In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy

from bellek.mhqa.llm import make_question_answer_func
from bellek.utils import set_seed, jprint
from bellek.musique.baseline import benchmark

set_seed(89)

In [3]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
df.set_index('id', inplace=True, drop=False)
df = df.loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [4]:
jerx_file = Path("../../data/raw/musique-evaluation/jerx-inferences/llama3-base.jsonl")
jerx_df = pd.read_json(jerx_file, lines=True)
jerx_df.head()

,id,paragraph_idx,paragraph_text,paragraph_title,is_supporting,text,input,generation
0,2hop__131818_161450,0,Maria Carrillo High School is a public high sc...,Maria Carrillo High School,False,# Maria Carrillo High School\nMaria Carrillo H...,[{'content': 'You are an excellent knowledge g...,Maria Carrillo High School | location | Santa ...
1,2hop__131818_161450,1,"Golestān Province (Persian: استان گلستان‎, Ost...",Golestan Province,True,# Golestan Province\nGolestān Province (Persia...,[{'content': 'You are an excellent knowledge g...,Golestan Province | location | north-east of I...
2,2hop__131818_161450,2,Voshmgir District () is a district (bakhsh) in...,Voshmgir District,True,# Voshmgir District\nVoshmgir District () is a...,[{'content': 'You are an excellent knowledge g...,"Voshmgir District | location | Aqqala County, ..."
3,2hop__131818_161450,3,52 Heroor is a village in the southern state o...,52 Heroor,False,# 52 Heroor\n52 Heroor is a village in the sou...,[{'content': 'You are an excellent knowledge g...,"52 Heroor | location | Karnataka, India\n52 He..."
4,2hop__131818_161450,4,Vennaimalai is a village of Karur District loc...,Vennaimalai,False,# Vennaimalai\nVennaimalai is a village of Kar...,[{'content': 'You are an excellent knowledge g...,Vennaimalai | location | Karur District\nVenna...


In [5]:
jerx_mapping = {(row['id'], row['paragraph_idx']): row['generation'] for _, row in jerx_df.iterrows()}

def extract_triplets(example: dict):
    example["triplets_str"] = [jerx_mapping[(example['id'], p['idx'])].strip() for p in example['paragraphs']]
    return example

In [6]:
df = df.apply(extract_triplets, axis=1)
print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers,triplets_str
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar],[Liliana Mumy | father | Bill Mumy\nBill Mumy ...
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County],[KAPE | broadcast frequency | 1550 AM\nKAPE | ...
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River],[Cabramatta Creek | location | Sydney\nCabrama...
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith],[Michael J. Barron | birth year | 1933\nMichae...
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster],[Perfect Night: Live in London | recorded by |...


In [7]:
COMPLETION_KWARGS={"temperature": 0.0, "max_tokens": 2048}
qa_func = make_question_answer_func("gpt-3.5-turbo", completion_kwargs=COMPLETION_KWARGS)

In [8]:
results = []

## Only paragraphs

In [9]:
_, scores = benchmark(df, qa_func, only_supporting=False)
results.append({**scores, "retrieval": "none", "context": "paragraphs"})
jprint(scores)

TypeError: benchmark() got an unexpected keyword argument 'only_supporting'

In [ ]:
_, scores = benchmark(df, qa_func, only_supporting=True)
results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs"})
jprint(scores)

  0%|          | 0/200 [00:00<?, ?it/s]

{
  "exact_match": 0.615,
  "f1": 0.7115097680097681,
  "fuzzy_match": 0.71
}


## Paragraphs + Triplets

In [ ]:
def enhance_paragraphs(row):
    paragraphs_with_triplets = []
    for p in row['paragraphs']:
        p = deepcopy(p) 
        triplets_str = str(jerx_mapping[(row['id'], p['idx'])])
        p['paragraph_text'] = '\n'.join([p['paragraph_text'], "# Entity-relation-entity triplets", triplets_str])
        paragraphs_with_triplets.append(p)
    row['paragraphs'] = paragraphs_with_triplets
    return row

df_paragraph_triplets = df.apply(enhance_paragraphs, axis=1) 
df_paragraph_triplets.head()
print(df_paragraph_triplets.iloc[0]['paragraphs'][2]['paragraph_text'])

Voshmgir District () is a district (bakhsh) in Aqqala County, Golestan Province, Iran. At the 2006 census, its population was 25,149, in 5,266 families. The District has one city: Anbar Olum. The District has two rural districts ("dehestan"): Mazraeh-ye Jonubi Rural District and Mazraeh-ye Shomali Rural District.
# Entity-relation-entity triplets
Voshmgir District | location | Aqqala County, Golestan Province, Iran
Voshmgir District | population | 25,149
Voshmgir District | population in families | 5,266
Voshmgir District | city | Anbar Olum
Voshmgir District | rural districts | Mazraeh-ye Jonubi Rural District, Mazraeh-ye Shomali Rural District


In [ ]:
_, scores = benchmark(df_paragraph_triplets, qa_func, only_supporting=False)
results.append({**scores, "retrieval": "none", "context": "paragraphs+triplets"})
jprint(scores)

  0%|          | 0/200 [00:00<?, ?it/s]

{
  "exact_match": 0.47,
  "f1": 0.5765849705849706,
  "fuzzy_match": 0.58
}


In [ ]:
_, scores = benchmark(df_paragraph_triplets, qa_func, only_supporting=True)
results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs+triplets"})
jprint(scores)

  0%|          | 0/200 [00:00<?, ?it/s]

{
  "exact_match": 0.58,
  "f1": 0.7009486069486072,
  "fuzzy_match": 0.71
}


## Only triplets

In [ ]:
def replace_paragraphs(row):
    paragraphs_with_triplets = []
    for p in row['paragraphs']:
        p = deepcopy(p) 
        triplets_str = str(jerx_mapping[(row['id'], p['idx'])])
        p['paragraph_text'] = '\n'.join(["# Entity-relation-entity triplets", triplets_str])
        paragraphs_with_triplets.append(p)
    row['paragraphs'] = paragraphs_with_triplets
    return row

df_only_triplets = df.apply(replace_paragraphs, axis=1) 
df_only_triplets.head()
print(df_only_triplets.iloc[0]['paragraphs'][2]['paragraph_text'])

# Entity-relation-entity triplets
Voshmgir District | location | Aqqala County, Golestan Province, Iran
Voshmgir District | population | 25,149
Voshmgir District | population in families | 5,266
Voshmgir District | city | Anbar Olum
Voshmgir District | rural districts | Mazraeh-ye Jonubi Rural District, Mazraeh-ye Shomali Rural District


In [ ]:
_, scores = benchmark(df_only_triplets, qa_func, only_supporting=False, ignore_errors=True)
results.append({**scores, "retrieval": "none", "context": "triplets"})
jprint(scores)

  0%|          | 0/200 [00:00<?, ?it/s]

{
  "exact_match": 0.425,
  "f1": 0.5433461538461539,
  "fuzzy_match": 0.55
}


In [ ]:
_, scores = benchmark(df_only_triplets, qa_func, only_supporting=True, ignore_errors=True)
results.append({**scores, "retrieval": "groundtruth", "context": "triplets"})
jprint(scores)

  0%|          | 0/200 [00:00<?, ?it/s]

Failed to answer the question 2hop__199513_13732
Unterminated string starting at: line 1 column 15 (char 14)
{
  "exact_match": 0.55,
  "f1": 0.6674594294594296,
  "fuzzy_match": 0.67
}


# Report

In [ ]:
report_df = pd.DataFrame.from_records(results, columns=['context', 'retrieval', 'exact_match', 'fuzzy_match', 'f1'])
report_df

,context,retrieval,exact_match,fuzzy_match,f1
0,paragraphs,none,0.500,0.595,0.582242
1,paragraphs,groundtruth,0.615,0.710,0.711510
2,paragraphs+triplets,none,0.470,0.580,0.576585
3,paragraphs+triplets,groundtruth,0.580,0.710,0.700949
4,triplets,none,0.425,0.550,0.543346
5,triplets,groundtruth,0.550,0.670,0.667459


In [ ]:
print(report_df[report_df['retrieval']=='none'].to_markdown(index=False))

| context             | retrieval   |   exact_match |   fuzzy_match |       f1 |
|:--------------------|:------------|--------------:|--------------:|---------:|
| paragraphs          | none        |         0.5   |         0.595 | 0.582242 |
| paragraphs+triplets | none        |         0.47  |         0.58  | 0.576585 |
| triplets            | none        |         0.425 |         0.55  | 0.543346 |


In [ ]:
print(report_df[report_df['retrieval']=='groundtruth'].to_markdown(index=False))

| context             | retrieval   |   exact_match |   fuzzy_match |       f1 |
|:--------------------|:------------|--------------:|--------------:|---------:|
| paragraphs          | groundtruth |         0.615 |          0.71 | 0.71151  |
| paragraphs+triplets | groundtruth |         0.58  |          0.71 | 0.700949 |
| triplets            | groundtruth |         0.55  |          0.67 | 0.667459 |
